In [ ]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape,  y_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=2045)

print(X_train.shape, y_train.shape, '/', X_valid.shape, y_valid.shape, '/', X_test.shape, y_test.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,shear_range=0.2, zoom_range=0.2,
                              horizontal_flip=True, vertical_flip=True, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size = 20)  ######### target_size
valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size = 20) ######### / batch_size
test_generator = test_datagen.flow(X_test, y_test, batch_size = 20)  ####### / binary or category ( class_mode = 'binary')

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (32, 32, 3))   ############ input shape 확인 ##########
conv_base.summary()

In [ ]:
set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block5_conv1':       ###### layer 이름 확인 #######
    set_trainable = True

  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

conv_base.summary()

In [ ]:
from keras import models, layers, optimizers

model = models.Sequential()
model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = 'softmax'))   ##### sigmoid or softmax ###########

model.summary()


model.compile(loss = 'categorical_crossentropy',  ############## binary or categorical
              optimizer = optimizers.Adam(lr = 0.000005),
              metrics = ['accuracy'])

In [ ]:
%%time

Hist = model.fit(X_train, y_train, epochs = 100, validation_data = (X_valid, y_valid))

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(Hist.history['loss']) + 1)
plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist.history['loss'])
plt.plot(epochs, Hist.history['val_loss'])
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()

epochs = range(1, len(Hist.history['loss']) + 1)
plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist.history['accuracy'])
plt.plot(epochs, Hist.history['val_accuracy'])
plt.title('Training & Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training accuracy', 'Validation accuracy'])
plt.grid()
plt.show()

* Accuracy Visualization

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)  

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

# google save

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/models/006_dogs_and_cats_VGG16.h5')

In [ ]:
!ls -l /content/drive/My\ Drive/Colab\ Notebooks/models

In [ ]:
from keras.models import load_model

model_google = load_model('/content/drive/My Drive/Colab Notebooks/models/006_dogs_and_cats_VGG16.h5')

In [ ]:
loss, accuracy = model_google.evaluate(test_generator,steps = 50) ########## steps 확인

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))